# --- Day 9: Marble Mania ---

You talk to the Elves while you wait for your navigation system to initialize. To pass the time, they introduce you to their favorite marble game.

The Elves play this game by taking turns arranging the marbles in a circle according to very particular rules. The marbles are numbered starting with 0 and increasing by 1 until every marble has a number.

First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.

Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.

However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.

For example, suppose there are 9 players. After the marble with value 0 is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:

```
[-] (0)
[1]  0 (1)
[2]  0 (2) 1 
[3]  0  2  1 (3)
[4]  0 (4) 2  1  3 
[5]  0  4  2 (5) 1  3 
[6]  0  4  2  5  1 (6) 3 
[7]  0  4  2  5  1  6  3 (7)
[8]  0 (8) 4  2  5  1  6  3  7 
[9]  0  8  4 (9) 2  5  1  6  3  7 
[1]  0  8  4  9  2(10) 5  1  6  3  7 
[2]  0  8  4  9  2 10  5(11) 1  6  3  7 
[3]  0  8  4  9  2 10  5 11  1(12) 6  3  7 
[4]  0  8  4  9  2 10  5 11  1 12  6(13) 3  7 
[5]  0  8  4  9  2 10  5 11  1 12  6 13  3(14) 7 
[6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7(15)
[7]  0(16) 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[8]  0 16  8(17) 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[9]  0 16  8 17  4(18) 9  2 10  5 11  1 12  6 13  3 14  7 15 
[1]  0 16  8 17  4 18  9(19) 2 10  5 11  1 12  6 13  3 14  7 15 
[2]  0 16  8 17  4 18  9 19  2(20)10  5 11  1 12  6 13  3 14  7 15 
[3]  0 16  8 17  4 18  9 19  2 20 10(21) 5 11  1 12  6 13  3 14  7 15 
[4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
[5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
```

The goal is to be the player with the highest score after the last marble is used up. Assuming the example above ends after the marble numbered 25, the winning score is 23+9=32 (because player 5 kept marble 23 and removed marble 9, while no other player got any points in this very short example game).

Here are a few more examples:

```
10 players; last marble is worth 1618 points: high score is 8317
13 players; last marble is worth 7999 points: high score is 146373
17 players; last marble is worth 1104 points: high score is 2764
21 players; last marble is worth 6111 points: high score is 54718
30 players; last marble is worth 5807 points: high score is 37305
```

What is the winning Elf's score?

In [24]:
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import cycle

In [3]:
@dataclass
class Game:
    players: int
    high_marble: int

In [4]:
def input_():
    return Game(430, 71588)

In [5]:
def test_inputs():
    yield Game(9, 25), 32
    yield Game(10, 1618), 8317
    yield Game(13, 7999), 146373
    yield Game(17, 1104), 2764
    yield Game(21, 6111), 54718
    yield Game(30, 5807), 37305

In [21]:
@dataclass
class Circle:
    state: list = field(init=False, repr=False, default_factory=lambda: [0])
    current_marble: int = field(init=False, default=0)
    
    def place_marble(self, marble):
        if marble % 23 == 0:
            raise RuntimeError('Adding marble that\'s a multiple of 23!')
        
        insertion_point = self.current_marble + 2
        if insertion_point > len(self.state):
            insertion_point = insertion_point - len(self.state)
        
        self.state.insert(insertion_point, marble)
        self.current_marble = insertion_point
        
    def remove_7cc(self):
        removal_point = self.current_marble - 7
        if removal_point < 0:
            removal_point = removal_point + len(self.state)
        
        removed = self.state.pop(removal_point)
        
        self.current_marble = removal_point % len(self.state)
        
        return removed

In [35]:
def run_game(players, max_marble):
    c = Circle()
    scores = defaultdict(lambda: 0)

    for player, marble in zip(cycle(range(players)), range(1, max_marble + 1)):
        if marble % 23 == 0:
            scores[player] += marble + c.remove_7cc()
        else:
            c.place_marble(marble)
    
    return max(scores.values())

In [36]:
run_game(9, 25)

32

In [38]:
for game, expected in test_inputs():
    assert run_game(game.players, game.high_marble) == expected

In [39]:
game = input_()
run_game(game.players, game.high_marble)

422748

# --- Part Two ---

Amused by the speed of your answer, the Elves are curious:

What would the new winning Elf's score be if the number of the last marble were 100 times larger?

In [40]:
game = input_()
run_game(game.players, game.high_marble * 100)

3412522480